<font size = "3"> Analyze the peerreview assignments using `SQLite` **DATABASE**

In [4]:

import sqlite3
import pandas as pd

conn = sqlite3.Connection("FinalDB.db")
cur = conn.cursor()

df = pd.read_csv("ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con=conn, if_exists='replace', index=False, method=None)

df_1 = pd.read_csv("ChicagoCrimeData.csv")
df_1.to_sql("CHICAGO_CRIME_DATA", con=conn, if_exists='replace', index=False, method=None)

df_2 = pd.read_csv("ChicagoPublicSchools.csv")
df_2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con=conn, if_exists='replace', index=False, method=None)

conn.close()


c:\users\sidik savutjan\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [15]:
%load_ext sql

%sql sqlite:///FinalDB.db
    
#%sql select * from CENSUS_DATA



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


UsageError: Line magic function `%tables` not found.


<font size = "3"> Retrieve the list of tables in the `SQLite` *database*.

In [25]:
%%sql

SELECT * FROM sqlite_master 
    WHERE type = 'table'


 * sqlite:///FinalDB.db
Done.


type,name,tbl_name,rootpage,sql
table,CENSUS_DATA,CENSUS_DATA,2,"CREATE TABLE ""CENSUS_DATA"" (""COMMUNITY_AREA_NUMBER"" REAL, ""COMMUNITY_AREA_NAME"" TEXT, ""PERCENT_OF_HOUSING_CROWDED"" REAL, ""PERCENT_HOUSEHOLDS_BELOW_POVERTY"" REAL, ""PERCENT_AGED_16__UNEMPLOYED"" REAL, ""PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA"" REAL, ""PERCENT_AGED_UNDER_18_OR_OVER_64"" REAL, ""PER_CAPITA_INCOME"" INTEGER, ""HARDSHIP_INDEX"" REAL)"
table,CHICAGO_CRIME_DATA,CHICAGO_CRIME_DATA,5,"CREATE TABLE ""CHICAGO_CRIME_DATA"" (""ID"" INTEGER, ""CASE_NUMBER"" TEXT, ""DATE"" TEXT, ""BLOCK"" TEXT, ""IUCR"" TEXT, ""PRIMARY_TYPE"" TEXT, ""DESCRIPTION"" TEXT, ""LOCATION_DESCRIPTION"" TEXT, ""ARREST"" INTEGER, ""DOMESTIC"" INTEGER, ""BEAT"" INTEGER, ""DISTRICT"" INTEGER, ""WARD"" REAL, ""COMMUNITY_AREA_NUMBER"" REAL, ""FBICODE"" TEXT, ""X_COORDINATE"" REAL, ""Y_COORDINATE"" REAL, ""YEAR"" INTEGER, ""LATITUDE"" REAL, ""LONGITUDE"" REAL, ""LOCATION"" TEXT)"
table,CHICAGO_PUBLIC_SCHOOLS,CHICAGO_PUBLIC_SCHOOLS,29,"CREATE TABLE ""CHICAGO_PUBLIC_SCHOOLS"" (""School_ID"" INTEGER, ""NAME_OF_SCHOOL"" TEXT, ""Elementary, Middle, or High School"" TEXT, ""Street_Address"" TEXT, ""City"" TEXT, ""State"" TEXT, ""ZIP_Code"" INTEGER, ""Phone_Number"" TEXT, ""Link"" TEXT, ""Network_Manager"" TEXT, ""Collaborative_Name"" TEXT, ""Adequate_Yearly_Progress_Made_"" TEXT, ""Track_Schedule"" TEXT, ""CPS_Performance_Policy_Status"" TEXT, ""CPS_Performance_Policy_Level"" TEXT, ""HEALTHY_SCHOOL_CERTIFIED"" TEXT, ""Safety_Icon"" TEXT, ""SAFETY_SCORE"" REAL, ""Family_Involvement_Icon"" TEXT, ""Family_Involvement_Score"" TEXT, ""Environment_Icon"" TEXT, ""Environment_Score"" REAL, ""Instruction_Icon"" TEXT, ""Instruction_Score"" REAL, ""Leaders_Icon"" TEXT, ""Leaders_Score"" TEXT, ""Teachers_Icon"" TEXT, ""Teachers_Score"" TEXT, ""Parent_Engagement_Icon"" TEXT, ""Parent_Engagement_Score"" TEXT, ""Parent_Environment_Icon"" TEXT, ""Parent_Environment_Score"" TEXT, ""AVERAGE_STUDENT_ATTENDANCE"" TEXT, ""Rate_of_Misconducts__per_100_students_"" REAL, ""Average_Teacher_Attendance"" TEXT, ""Individualized_Education_Program_Compliance_Rate"" TEXT, ""Pk_2_Literacy__"" TEXT, ""Pk_2_Math__"" TEXT, ""Gr3_5_Grade_Level_Math__"" TEXT, ""Gr3_5_Grade_Level_Read__"" TEXT, ""Gr3_5_Keep_Pace_Read__"" TEXT, ""Gr3_5_Keep_Pace_Math__"" TEXT, ""Gr6_8_Grade_Level_Math__"" TEXT, ""Gr6_8_Grade_Level_Read__"" TEXT, ""Gr6_8_Keep_Pace_Math_"" TEXT, ""Gr6_8_Keep_Pace_Read__"" TEXT, ""Gr_8_Explore_Math__"" TEXT, ""Gr_8_Explore_Read__"" TEXT, ""ISAT_Exceeding_Math__"" REAL, ""ISAT_Exceeding_Reading__"" REAL, ""ISAT_Value_Add_Math"" REAL, ""ISAT_Value_Add_Read"" REAL, ""ISAT_Value_Add_Color_Math"" TEXT, ""ISAT_Value_Add_Color_Read"" TEXT, ""Students_Taking__Algebra__"" TEXT, ""Students_Passing__Algebra__"" TEXT, ""9th Grade EXPLORE (2009)"" TEXT, ""9th Grade EXPLORE (2010)"" TEXT, ""10th Grade PLAN (2009)"" TEXT, ""10th Grade PLAN (2010)"" TEXT, ""Net_Change_EXPLORE_and_PLAN"" TEXT, ""11th Grade Average ACT (2011)"" TEXT, ""Net_Change_PLAN_and_ACT"" TEXT, ""College_Eligibility__"" TEXT, ""Graduation_Rate__"" TEXT, ""College_Enrollment_Rate__"" TEXT, ""COLLEGE_ENROLLMENT"" INTEGER, ""General_Services_Route"" INTEGER, ""Freshman_on_Track_Rate__"" TEXT, ""X_COORDINATE"" REAL, ""Y_COORDINATE"" REAL, ""Latitude"" REAL, ""Longitude"" REAL, ""COMMUNITY_AREA_NUMBER"" INTEGER, ""COMMUNITY_AREA_NAME"" TEXT, ""Ward"" INTEGER, ""Police_District"" INTEGER, ""Location"" TEXT)"


**P1**: find the total number of crimes in the CRIME table.

In [26]:
%%sql

SELECT COUNT(*) as total_num_crime FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


total_num_crime
533


**P2**: list community areas with **per capita** income less than 11000.

In [27]:
%%sql 

SELECT community_area_name, per_capita_income FROM CENSUS_DATA
    WHERE per_capita_income < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


**P3**: list all case numbers for crimes *involving minors*.

In [28]:
%%sql 

SELECT case_number, DESCRIPTION FROM CHICAGO_CRIME_DATA

    WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


**P4**: list all kidnapping crimes involving a child.

In [29]:
%%sql

SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION 
    FROM CHICAGO_CRIME_DATA
        WHERE PRIMARY_TYPE LIKE 'KIDNAPPING';


 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


**P5**: what kind of crimes were recorded at *schools*.

In [33]:
%%sql

SELECT DISTINCT PRIMARY_TYPE FROM CHICAGO_CRIME_DATA
    WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'
        ORDER BY PRIMARY_TYPE;

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPASS
NARCOTICS
PUBLIC PEACE VIOLATION


**P6**: list the *average safety score* for all types of schools

In [35]:
%%sql

SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS AVG_SAFETY_ASCORE
    FROM CHICAGO_PUBLIC_SCHOOLS
    GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG_SAFETY_ASCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


**P7**: list 5 community areas wiht highest % of households *below poverty*.

In [36]:
%%sql

SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
    FROM CENSUS_DATA
        ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
            LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


**P8**: which community area(number) is most crime prone?

In [42]:
%%sql 

DELETE FROM CHICAGO_CRIME_DATA
    WHERE COMMUNITY_AREA_NUMBER IS NULL;

SELECT COMMUNITY_AREA_NUMBER, COUNT(*) as TOTAL_CRIME_NUM 
    FROM CHICAGO_CRIME_DATA
        GROUP BY COMMUNITY_AREA_NUMBER
            ORDER BY TOTAL_CRIME_NUM DESC LIMIT 1;

 * sqlite:///FinalDB.db
0 rows affected.
Done.


COMMUNITY_AREA_NUMBER,TOTAL_CRIME_NUM
25.0,43


**P9**: use a `sub_query` to find the name of the community area with highest *hardship index*.

In [43]:
%%sql

SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX 
    FROM CENSUS_DATA
        WHERE HARDSHIP_INDEX = 
            (select max(HARDSHIP_INDEX) from CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


**P10**: use a `sub_query` to determine the community area name with most number of crimes.

In [44]:
%%sql 

SELECT COMMUNITY_AREA_NAME
    FROM CENSUS_DATA
        WHERE COMMUNITY_AREA_NUMBER =
            (select community_area_number
                from CHICAGO_CRIME_DATA
                    group by community_area_number
                        order by count(community_area_number) DESC LIMIT 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
